## https://www.kaggle.com/imoore/60k-stack-overflow-questions-with-quality-rate
## We recommend you to reach above websites and click the "new notebook" inside, turn on the TPU accelerator.  The Kaggle enviromnet would be preferable to run our code. If you want to run our code locally, please download the dataset from above website, and change the first few lines in code to refer to your directory with local dataset.

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/60k-stack-overflow-questions-with-quality-rate/valid.csv
/kaggle/input/60k-stack-overflow-questions-with-quality-rate/train.csv


In [2]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
import os
import sys
import time
import tensorflow as tf
from tensorflow import keras
import re

print(sys.version_info)
for module in mpl, np, pd, sklearn, tf, keras:
    print(module.__name__, module.__version__)

sys.version_info(major=3, minor=7, micro=10, releaselevel='final', serial=0)
matplotlib 3.4.1
numpy 1.19.5
pandas 1.2.3
sklearn 0.24.1
tensorflow 2.4.1
tensorflow.keras 2.4.0


# Data Preprocessing


In [3]:
stack_data1 = pd.read_csv(r'/kaggle/input/60k-stack-overflow-questions-with-quality-rate/valid.csv')
stack_data2 = pd.read_csv(r'/kaggle/input/60k-stack-overflow-questions-with-quality-rate/train.csv')
stack_data = pd.concat([stack_data1,stack_data2])
stack_data.columns

Index(['Id', 'Title', 'Body', 'Tags', 'CreationDate', 'Y'], dtype='object')

In [4]:
# Drop the NAN(useless information) from dataset
stack_data_f = stack_data.dropna(subset=['Y'])
# Combine the Title and Body part of comment as the input X 
stack_data_f['text'] = stack_data_f.Title +': '+ stack_data_f.Body + stack_data_f.Tags

In [5]:
# Transform the label into one-hot vector
Y = stack_data_f.Y
format_new = {'LQ_CLOSE':0, 'HQ':1, 'LQ_EDIT':2}
Y = np.array(Y.apply(lambda x: format_new[x]))

Y_onehot = []
formatq = [[1, 0, 0], [0, 1, 0], [0, 0, 1]]
for y in Y:
    Y_onehot.append(formatq[y])


In [6]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

# Split the train and test dataset with test-size = 0.2
X_train, X_test, y_train, y_test = train_test_split(stack_data_f.text, np.array(Y_onehot), test_size=0.2, random_state=0 )

maxfeatures = 20000 # Use 20000 words with most frequnences


# Use tokenizer to encode the data from text to number
tok = tf.keras.preprocessing.text.Tokenizer(num_words = maxfeatures)
tok.fit_on_texts(stack_data_f.text)
X_train = tok.texts_to_sequences(X_train)
X_test = tok.texts_to_sequences(X_test)

# Use the mean length of data as the restriction length
sentence_all = tok.texts_to_sequences(stack_data_f.text)
len_estimate = []
for x in sentence_all:
    len_estimate.append(len(x))

MAX_LEN = int(np.mean(len_estimate)) # Restrict the length of a comment into fixed words number

# Use padding to uniform the sentences into same length
X_train = tf.keras.preprocessing.sequence.pad_sequences(X_train, maxlen=MAX_LEN, padding='post')
X_test = tf.keras.preprocessing.sequence.pad_sequences(X_test, maxlen=MAX_LEN, padding='post')

In [7]:
from keras import layers

def model_all(BATCH_SIZE, net_units, model_sel, EPOCHS = 20):
    # Instantiate a distribution strategy of tpu in Kaggle
    # If you don't want to use Kaggle's TPU, you can ignore these lines.
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

    with tpu_strategy.scope():
        inputs = tf.keras.Input(shape=(None,), dtype="int32")
        x = layers.Embedding(20000, 64)(inputs)
        if model_sel is 'GRU':
            x = layers.Bidirectional(layers.GRU(net_units, return_sequences=True))(x)
            x = layers.Bidirectional(layers.GRU(64))(x)
        elif model_sel is 'LSTM':
            x = layers.Bidirectional(layers.LSTM(net_units, return_sequences=True))(x)
            x = layers.Bidirectional(layers.LSTM(64))(x)
        else:
            x = layers.Bidirectional(layers.GRU(net_units, return_sequences=True))(x)
            x = layers.Bidirectional(layers.GRU(64))(x)
        x = layers.Dense(32,  kernel_regularizer=keras.regularizers.l2(0.01))(x)
        x = layers.Dropout(0.2)(x)
        outputs = layers.Dense(3, activation='softmax')(x)
        model = tf.keras.Model(inputs, outputs)
#         model.summary()
        model.compile(loss='categorical_crossentropy', optimizer='adam' ,metrics='categorical_accuracy')
        history = model.fit(X_train, y_train, batch_size=BATCH_SIZE, epochs=EPOCHS, validation_split = 0.2, verbose=0, 
                            callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_categorical_accuracy', 
                                                                          patience=3, mode = 'auto')])
        loss, accuracy = model.evaluate(X_test, y_test)
        return accuracy

In [8]:
model_sele_all = ['LSTM', 'GRU']
batch_size_all = [100, 50, 20]
net_units_all = [256, 128, 64]
res_search = []
for model in model_sele_all:
    for net_units in net_units_all:
        for batch_size in batch_size_all:
            acc = model_all(batch_size, net_units, model)
            print([acc, model, batch_size, net_units])
            res_search.append([acc, model, batch_size, net_units])

375/375 [==============================] - 8s 15ms/step - loss: 0.2794 - categorical_accuracy: 0.9083
[0.9083333015441895, 'LSTM', 100, 256]
375/375 [==============================] - 7s 14ms/step - loss: 0.3046 - categorical_accuracy: 0.9076
[0.9075832962989807, 'LSTM', 50, 256]
375/375 [==============================] - 8s 15ms/step - loss: 0.3574 - categorical_accuracy: 0.8913
[0.8912500143051147, 'LSTM', 20, 256]
375/375 [==============================] - 7s 14ms/step - loss: 0.3461 - categorical_accuracy: 0.8958
[0.8958333134651184, 'LSTM', 100, 128]
375/375 [==============================] - 8s 15ms/step - loss: 0.3426 - categorical_accuracy: 0.9019
[0.9019166827201843, 'LSTM', 50, 128]
375/375 [==============================] - 7s 14ms/step - loss: 0.2679 - categorical_accuracy: 0.9065
[0.906499981880188, 'LSTM', 20, 128]
375/375 [==============================] - 7s 13ms/step - loss: 0.2745 - categorical_accuracy: 0.9032
[0.903166651725769, 'LSTM', 100, 64]
375/375 [===========